# Portfolio Backtesting and Performance Analysis

This notebook covers:
- Historical backtesting of optimized portfolios
- Performance attribution analysis
- Risk-adjusted performance metrics
- Comparison with benchmark portfolios
- Sensitivity analysis and stress testing

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from datetime import datetime, timedelta

# Performance analysis libraries
from scipy import stats
import empyrical as ep
from sklearn.metrics import mean_squared_error

# Custom modules
from src.data.loader import DataLoader
from src.backtesting.engine import BacktestEngine
from src.utils.logger import setup_logger

warnings.filterwarnings('ignore')
logger = setup_logger(__name__)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Backtesting libraries loaded successfully")

## Load Data and Portfolio Recommendation

In [ ]:
# Load market data
loader = DataLoader()
data = loader.load_data(['TSLA', 'BND', 'SPY'], '2015-07-01', '2025-07-31')
prices = data['prices']
returns = data['returns']

# Load portfolio recommendation
try:
    with open('../data/portfolio_recommendation.pkl', 'rb') as f:
        portfolio_rec = pickle.load(f)
    print("✓ Portfolio recommendation loaded")
    
    print(f"\nRecommended Strategy: {portfolio_rec['strategy']}")
    print(f"Expected Return: {portfolio_rec['expected_return']*100:.2f}%")
    print(f"Expected Volatility: {portfolio_rec['expected_volatility']*100:.2f}%")
    print(f"Sharpe Ratio: {portfolio_rec['sharpe_ratio']:.3f}")
    
    recommended_weights = portfolio_rec['weights']
    print(f"\nWeights:")
    for asset, weight in recommended_weights.items():
        print(f"  {asset}: {weight*100:.2f}%")
        
except FileNotFoundError:
    print("✗ Portfolio recommendation not found. Using default weights.")
    recommended_weights = {'TSLA': 0.4, 'BND': 0.3, 'SPY': 0.3}
    portfolio_rec = None

# Asset information
assets = ['TSLA', 'BND', 'SPY']
print(f"\nBacktesting period: {prices.index.min()} to {prices.index.max()}")
print(f"Total observations: {len(prices)}")

## Backtesting Framework Setup

In [ ]:
class PortfolioBacktester:
    """
    Comprehensive portfolio backtesting framework
    """
    
    def __init__(self, prices_data, returns_data, initial_capital=100000):
        self.prices = prices_data
        self.returns = returns_data
        self.initial_capital = initial_capital
        self.assets = list(prices_data.columns)
        
    def calculate_portfolio_returns(self, weights, rebalance_freq='M'):
        """
        Calculate portfolio returns with rebalancing
        """
        # Convert weights to pandas Series if dict
        if isinstance(weights, dict):
            weights = pd.Series(weights)
        
        # Align weights with assets
        weights = weights.reindex(self.assets, fill_value=0)
        
        if rebalance_freq == 'B':  # Buy and hold
            # Simple buy and hold strategy
            portfolio_returns = (self.returns * weights).sum(axis=1)
        else:
            # Rebalancing strategy
            portfolio_returns = []
            current_weights = weights.copy()
            
            # Group by rebalancing frequency
            grouped = self.returns.groupby(pd.Grouper(freq=rebalance_freq))
            
            for period, period_returns in grouped:
                if len(period_returns) == 0:
                    continue
                    
                # Calculate returns for this period
                period_portfolio_returns = (period_returns * current_weights).sum(axis=1)
                portfolio_returns.extend(period_portfolio_returns.tolist())
                
                # Rebalance at end of period (reset to target weights)
                current_weights = weights.copy()
            
            portfolio_returns = pd.Series(portfolio_returns, index=self.returns.index[:len(portfolio_returns)])
        
        return portfolio_returns
    
    def calculate_performance_metrics(self, portfolio_returns, benchmark_returns=None, risk_free_rate=0.02):
        """
        Calculate comprehensive performance metrics
        """
        # Basic metrics
        total_return = (1 + portfolio_returns).prod() - 1
        annualized_return = (1 + total_return) ** (252 / len(portfolio_returns)) - 1
        annualized_volatility = portfolio_returns.std() * np.sqrt(252)
        
        # Risk-adjusted metrics
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
        
        # Drawdown analysis
        cumulative_returns = (1 + portfolio_returns).cumprod()
        rolling_max = cumulative_returns.expanding().max()
        drawdown = (cumulative_returns - rolling_max) / rolling_max
        max_drawdown = drawdown.min()
        
        # Calmar ratio
        calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown != 0 else np.inf
        
        # VaR and CVaR
        var_95 = np.percentile(portfolio_returns, 5)
        var_99 = np.percentile(portfolio_returns, 1)
        cvar_95 = portfolio_returns[portfolio_returns <= var_95].mean()
        cvar_99 = portfolio_returns[portfolio_returns <= var_99].mean()
        
        # Skewness and Kurtosis
        skewness = portfolio_returns.skew()
        kurtosis = portfolio_returns.kurtosis()
        
        # Win rate
        win_rate = (portfolio_returns > 0).mean()
        
        metrics = {
            'Total Return': total_return,
            'Annualized Return': annualized_return,
            'Annualized Volatility': annualized_volatility,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown': max_drawdown,
            'Calmar Ratio': calmar_ratio,
            'VaR 95%': var_95,
            'VaR 99%': var_99,
            'CVaR 95%': cvar_95,
            'CVaR 99%': cvar_99,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'Win Rate': win_rate
        }
        
        # Benchmark comparison if provided
        if benchmark_returns is not None:
            # Align returns
            aligned_portfolio, aligned_benchmark = portfolio_returns.align(benchmark_returns, join='inner')
            
            # Information ratio
            excess_returns = aligned_portfolio - aligned_benchmark
            information_ratio = excess_returns.mean() / excess_returns.std() * np.sqrt(252)
            
            # Beta and Alpha
            covariance = np.cov(aligned_portfolio, aligned_benchmark)[0, 1]
            benchmark_variance = aligned_benchmark.var()
            beta = covariance / benchmark_variance
            
            benchmark_annual_return = (1 + aligned_benchmark).prod() ** (252 / len(aligned_benchmark)) - 1
            alpha = annualized_return - (risk_free_rate + beta * (benchmark_annual_return - risk_free_rate))
            
            # Tracking error
            tracking_error = excess_returns.std() * np.sqrt(252)
            
            metrics.update({
                'Information Ratio': information_ratio,
                'Beta': beta,
                'Alpha': alpha,
                'Tracking Error': tracking_error
            })
        
        return metrics
    
    def rolling_performance(self, portfolio_returns, window=252):
        """
        Calculate rolling performance metrics
        """
        rolling_return = portfolio_returns.rolling(window).apply(lambda x: (1 + x).prod() - 1)
        rolling_volatility = portfolio_returns.rolling(window).std() * np.sqrt(252)
        rolling_sharpe = (rolling_return * 252 / window - 0.02) / rolling_volatility
        
        return {
            'rolling_return': rolling_return,
            'rolling_volatility': rolling_volatility,
            'rolling_sharpe': rolling_sharpe
        }

# Initialize backtester
backtester = PortfolioBacktester(prices, returns)
print("Backtesting framework initialized")

## Portfolio Strategy Backtesting

In [ ]:
# Define portfolio strategies to backtest
strategies = {
    'Optimized Portfolio': recommended_weights,
    'Equal Weight': {'TSLA': 1/3, 'BND': 1/3, 'SPY': 1/3},
    '60/40 Traditional': {'TSLA': 0.0, 'BND': 0.4, 'SPY': 0.6},
    'Conservative': {'TSLA': 0.1, 'BND': 0.6, 'SPY': 0.3},
    'Aggressive': {'TSLA': 0.6, 'BND': 0.1, 'SPY': 0.3}
}

# Backtest all strategies
backtest_results = {}
rebalance_frequency = 'M'  # Monthly rebalancing

print("BACKTESTING PORTFOLIO STRATEGIES")
print("=" * 60)
print(f"Rebalancing Frequency: {rebalance_frequency}")
print(f"Backtesting Period: {returns.index.min()} to {returns.index.max()}")

for strategy_name, weights in strategies.items():
    print(f"\nBacktesting {strategy_name}...")
    
    # Calculate portfolio returns
    portfolio_returns = backtester.calculate_portfolio_returns(weights, rebalance_frequency)
    
    # Use SPY as benchmark
    benchmark_returns = returns['SPY']
    
    # Calculate performance metrics
    metrics = backtester.calculate_performance_metrics(
        portfolio_returns, benchmark_returns
    )
    
    # Calculate rolling metrics
    rolling_metrics = backtester.rolling_performance(portfolio_returns)
    
    # Store results
    backtest_results[strategy_name] = {
        'returns': portfolio_returns,
        'metrics': metrics,
        'rolling': rolling_metrics,
        'weights': weights
    }
    
    print(f"  Total Return: {metrics['Total Return']*100:.2f}%")
    print(f"  Annualized Return: {metrics['Annualized Return']*100:.2f}%")
    print(f"  Volatility: {metrics['Annualized Volatility']*100:.2f}%")
    print(f"  Sharpe Ratio: {metrics['Sharpe Ratio']:.3f}")
    print(f"  Max Drawdown: {metrics['Max Drawdown']*100:.2f}%")

print("\nBacktesting completed successfully!")

## Performance Comparison and Analysis

In [ ]:
# Create comprehensive performance comparison
performance_data = []

for strategy_name, results in backtest_results.items():
    metrics = results['metrics']
    row = [
        strategy_name,
        metrics['Total Return'] * 100,
        metrics['Annualized Return'] * 100,
        metrics['Annualized Volatility'] * 100,
        metrics['Sharpe Ratio'],
        metrics['Max Drawdown'] * 100,
        metrics['Calmar Ratio'],
        metrics['Information Ratio'],
        metrics['Beta'],
        metrics['Alpha'] * 100,
        metrics['Win Rate'] * 100
    ]
    performance_data.append(row)

columns = [
    'Strategy', 'Total Return (%)', 'Annual Return (%)', 'Volatility (%)',
    'Sharpe Ratio', 'Max Drawdown (%)', 'Calmar Ratio', 'Information Ratio',
    'Beta', 'Alpha (%)', 'Win Rate (%)'
]

performance_df = pd.DataFrame(performance_data, columns=columns)
performance_df = performance_df.sort_values('Sharpe Ratio', ascending=False)

print("COMPREHENSIVE PERFORMANCE COMPARISON")
print("=" * 100)
print(performance_df.round(3).to_string(index=False))

# Identify best performing strategy
best_strategy = performance_df.iloc[0]['Strategy']
print(f"\nBest Performing Strategy (by Sharpe Ratio): {best_strategy}")

In [ ]:
# Visualize performance comparison
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Portfolio Strategy Performance Comparison', fontsize=16, fontweight='bold')

# 1. Cumulative Returns
ax1 = axes[0, 0]
for strategy_name, results in backtest_results.items():
    cumulative_returns = (1 + results['returns']).cumprod()
    ax1.plot(cumulative_returns.index, cumulative_returns, 
             label=strategy_name, linewidth=2)

ax1.set_title('Cumulative Returns')
ax1.set_ylabel('Cumulative Return')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# 2. Risk-Return Scatter
ax2 = axes[0, 1]
for strategy_name, results in backtest_results.items():
    metrics = results['metrics']
    ax2.scatter(metrics['Annualized Volatility']*100, 
               metrics['Annualized Return']*100,
               s=100, label=strategy_name)

ax2.set_xlabel('Volatility (%)')
ax2.set_ylabel('Annual Return (%)')
ax2.set_title('Risk-Return Profile')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Drawdown Analysis
ax3 = axes[0, 2]
for strategy_name, results in backtest_results.items():
    portfolio_returns = results['returns']
    cumulative_returns = (1 + portfolio_returns).cumprod()
    rolling_max = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - rolling_max) / rolling_max
    
    ax3.fill_between(drawdown.index, drawdown*100, 0, alpha=0.7, label=strategy_name)

ax3.set_title('Drawdown Analysis')
ax3.set_ylabel('Drawdown (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Rolling Sharpe Ratio
ax4 = axes[1, 0]
for strategy_name, results in backtest_results.items():
    rolling_sharpe = results['rolling']['rolling_sharpe']
    ax4.plot(rolling_sharpe.index, rolling_sharpe, 
             label=strategy_name, linewidth=2, alpha=0.8)

ax4.set_title('Rolling Sharpe Ratio (1-Year)')
ax4.set_ylabel('Sharpe Ratio')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Performance Metrics Bar Chart
ax5 = axes[1, 1]
metrics_to_plot = ['Sharpe Ratio', 'Calmar Ratio', 'Information Ratio']
x = np.arange(len(strategies))
width = 0.25

for i, metric in enumerate(metrics_to_plot):
    values = [backtest_results[strategy]['metrics'][metric] for strategy in strategies.keys()]
    ax5.bar(x + i*width, values, width, label=metric, alpha=0.8)

ax5.set_xlabel('Strategy')
ax5.set_ylabel('Ratio')
ax5.set_title('Risk-Adjusted Performance Metrics')
ax5.set_xticks(x + width)
ax5.set_xticklabels(list(strategies.keys()), rotation=45, ha='right')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Return Distribution
ax6 = axes[1, 2]
for strategy_name, results in backtest_results.items():
    portfolio_returns = results['returns']
    ax6.hist(portfolio_returns*100, bins=50, alpha=0.6, label=strategy_name, density=True)

ax6.set_xlabel('Daily Returns (%)')
ax6.set_ylabel('Density')
ax6.set_title('Return Distribution')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Time-Based Performance Analysis

In [ ]:
# Analyze performance by different time periods
def analyze_period_performance(returns_series, start_date, end_date, period_name):
    """
    Analyze performance for a specific time period
    """
    period_returns = returns_series[(returns_series.index >= start_date) & 
                                   (returns_series.index <= end_date)]
    
    if len(period_returns) == 0:
        return None
    
    total_return = (1 + period_returns).prod() - 1
    volatility = period_returns.std() * np.sqrt(252)
    sharpe = (total_return * 252 / len(period_returns) - 0.02) / volatility
    max_dd = ((1 + period_returns).cumprod() / (1 + period_returns).cumprod().expanding().max() - 1).min()
    
    return {
        'Period': period_name,
        'Total Return': total_return * 100,
        'Volatility': volatility * 100,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': max_dd * 100
    }

# Define analysis periods
analysis_periods = [
    ('2015-07-01', '2017-12-31', '2015-2017'),
    ('2018-01-01', '2019-12-31', '2018-2019'),
    ('2020-01-01', '2021-12-31', '2020-2021 (COVID)'),
    ('2022-01-01', '2023-12-31', '2022-2023'),
    ('2024-01-01', '2025-07-31', '2024-2025')
]

# Analyze each strategy by time period
period_analysis = {}

for strategy_name, results in backtest_results.items():
    strategy_periods = []
    
    for start_date, end_date, period_name in analysis_periods:
        period_result = analyze_period_performance(
            results['returns'], start_date, end_date, period_name
        )
        if period_result:
            strategy_periods.append(period_result)
    
    period_analysis[strategy_name] = pd.DataFrame(strategy_periods)

# Display period analysis for optimized portfolio
optimized_strategy = 'Optimized Portfolio'
if optimized_strategy in period_analysis:
    print(f"PERIOD ANALYSIS - {optimized_strategy}")
    print("=" * 60)
    print(period_analysis[optimized_strategy].round(2).to_string(index=False))

# Compare strategies across periods
print(f"\nSHARPE RATIO COMPARISON ACROSS PERIODS")
print("=" * 60)

sharpe_comparison = {}
for strategy_name, period_df in period_analysis.items():
    sharpe_comparison[strategy_name] = period_df.set_index('Period')['Sharpe Ratio']

sharpe_df = pd.DataFrame(sharpe_comparison)
print(sharpe_df.round(3))

# Visualize period performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Time-Based Performance Analysis', fontsize=16, fontweight='bold')

# Sharpe ratio by period
ax1 = axes[0, 0]
sharpe_df.plot(kind='bar', ax=ax1, alpha=0.8)
ax1.set_title('Sharpe Ratio by Period')
ax1.set_ylabel('Sharpe Ratio')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Return by period
ax2 = axes[0, 1]
return_comparison = {}
for strategy_name, period_df in period_analysis.items():
    return_comparison[strategy_name] = period_df.set_index('Period')['Total Return']

return_df = pd.DataFrame(return_comparison)
return_df.plot(kind='bar', ax=ax2, alpha=0.8)
ax2.set_title('Total Return by Period (%)')
ax2.set_ylabel('Total Return (%)')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# Volatility by period
ax3 = axes[1, 0]
vol_comparison = {}
for strategy_name, period_df in period_analysis.items():
    vol_comparison[strategy_name] = period_df.set_index('Period')['Volatility']

vol_df = pd.DataFrame(vol_comparison)
vol_df.plot(kind='bar', ax=ax3, alpha=0.8)
ax3.set_title('Volatility by Period (%)')
ax3.set_ylabel('Volatility (%)')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# Max drawdown by period
ax4 = axes[1, 1]
dd_comparison = {}
for strategy_name, period_df in period_analysis.items():
    dd_comparison[strategy_name] = period_df.set_index('Period')['Max Drawdown']

dd_df = pd.DataFrame(dd_comparison)
dd_df.plot(kind='bar', ax=ax4, alpha=0.8)
ax4.set_title('Max Drawdown by Period (%)')
ax4.set_ylabel('Max Drawdown (%)')
ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax4.grid(True, alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Stress Testing and Scenario Analysis

In [ ]:
# Stress testing framework
class StressTester:
    """
    Portfolio stress testing and scenario analysis
    """
    
    def __init__(self, returns_data, weights):
        self.returns = returns_data
        self.weights = pd.Series(weights) if isinstance(weights, dict) else weights
        self.portfolio_returns = (returns_data * self.weights).sum(axis=1)
        
    def tail_risk_analysis(self):
        """
        Analyze tail risk scenarios
        """
        # Identify worst performing days
        worst_days = self.portfolio_returns.nsmallest(10)
        best_days = self.portfolio_returns.nlargest(10)
        
        # Calculate tail statistics
        var_95 = np.percentile(self.portfolio_returns, 5)
        var_99 = np.percentile(self.portfolio_returns, 1)
        var_999 = np.percentile(self.portfolio_returns, 0.1)
        
        cvar_95 = self.portfolio_returns[self.portfolio_returns <= var_95].mean()
        cvar_99 = self.portfolio_returns[self.portfolio_returns <= var_99].mean()
        
        return {
            'worst_days': worst_days,
            'best_days': best_days,
            'var_95': var_95,
            'var_99': var_99,
            'var_999': var_999,
            'cvar_95': cvar_95,
            'cvar_99': cvar_99
        }
    
    def market_crash_scenarios(self):
        """
        Analyze performance during market crashes
        """
        # Define crash periods (approximate)
        crash_periods = {
            'COVID Crash': ('2020-02-20', '2020-03-23'),
            'Dec 2018 Selloff': ('2018-10-01', '2018-12-24'),
            'Early 2022 Correction': ('2022-01-01', '2022-03-15'),
            'Aug 2015 Correction': ('2015-08-01', '2015-08-31')
        }
        
        crash_performance = {}
        
        for crash_name, (start_date, end_date) in crash_periods.items():
            try:
                crash_returns = self.portfolio_returns[
                    (self.portfolio_returns.index >= start_date) & 
                    (self.portfolio_returns.index <= end_date)
                ]
                
                if len(crash_returns) > 0:
                    total_return = (1 + crash_returns).prod() - 1
                    max_dd = ((1 + crash_returns).cumprod() / 
                             (1 + crash_returns).cumprod().expanding().max() - 1).min()
                    
                    crash_performance[crash_name] = {
                        'total_return': total_return * 100,
                        'max_drawdown': max_dd * 100,
                        'worst_day': crash_returns.min() * 100,
                        'volatility': crash_returns.std() * np.sqrt(252) * 100
                    }
            except:
                continue
        
        return crash_performance
    
    def monte_carlo_stress_test(self, n_simulations=10000, time_horizon=252):
        """
        Monte Carlo stress testing
        """
        # Calculate portfolio statistics
        mean_return = self.portfolio_returns.mean()
        volatility = self.portfolio_returns.std()
        
        # Generate random scenarios
        np.random.seed(42)
        simulated_returns = np.random.normal(mean_return, volatility, 
                                           (n_simulations, time_horizon))
        
        # Calculate final portfolio values
        final_values = np.prod(1 + simulated_returns, axis=1)
        
        # Calculate statistics
        percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
        percentile_values = np.percentile(final_values, percentiles)
        
        return {
            'percentiles': dict(zip(percentiles, percentile_values)),
            'probability_of_loss': np.mean(final_values < 1) * 100,
            'expected_value': np.mean(final_values),
            'worst_case_1pct': np.percentile(final_values, 1)
        }

# Perform stress testing on optimized portfolio
stress_tester = StressTester(returns, recommended_weights)

print("STRESS TESTING ANALYSIS")
print("=" * 60)

# Tail risk analysis
tail_risk = stress_tester.tail_risk_analysis()

print(f"\nTAIL RISK ANALYSIS:")
print(f"-" * 30)
print(f"VaR 95%: {tail_risk['var_95']*100:.2f}%")
print(f"VaR 99%: {tail_risk['var_99']*100:.2f}%")
print(f"VaR 99.9%: {tail_risk['var_999']*100:.2f}%")
print(f"CVaR 95%: {tail_risk['cvar_95']*100:.2f}%")
print(f"CVaR 99%: {tail_risk['cvar_99']*100:.2f}%")

print(f"\nWorst 5 Days:")
for date, return_val in tail_risk['worst_days'].head().items():
    print(f"  {date.strftime('%Y-%m-%d')}: {return_val*100:.2f}%")

# Market crash analysis
crash_performance = stress_tester.market_crash_scenarios()

print(f"\nMARKET CRASH PERFORMANCE:")
print(f"-" * 30)
for crash_name, performance in crash_performance.items():
    print(f"\n{crash_name}:")
    print(f"  Total Return: {performance['total_return']:.2f}%")
    print(f"  Max Drawdown: {performance['max_drawdown']:.2f}%")
    print(f"  Worst Day: {performance['worst_day']:.2f}%")
    print(f"  Volatility: {performance['volatility']:.2f}%")

# Monte Carlo stress test
mc_stress = stress_tester.monte_carlo_stress_test()

print(f"\nMONTE CARLO STRESS TEST (1-Year Horizon):")
print(f"-" * 30)
print(f"Expected Portfolio Value: {mc_stress['expected_value']:.3f}")
print(f"Probability of Loss: {mc_stress['probability_of_loss']:.2f}%")
print(f"Worst Case (1%): {mc_stress['worst_case_1pct']:.3f}")

print(f"\nPercentile Distribution:")
for percentile, value in mc_stress['percentiles'].items():
    print(f"  {percentile}th percentile: {value:.3f}")

In [ ]:
# Visualize stress testing results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Stress Testing and Risk Analysis', fontsize=16, fontweight='bold')

# 1. Return distribution with VaR levels
ax1 = axes[0, 0]
portfolio_returns = stress_tester.portfolio_returns
ax1.hist(portfolio_returns*100, bins=50, alpha=0.7, density=True, color='skyblue')
ax1.axvline(tail_risk['var_95']*100, color='orange', linestyle='--', linewidth=2, label='VaR 95%')
ax1.axvline(tail_risk['var_99']*100, color='red', linestyle='--', linewidth=2, label='VaR 99%')
ax1.axvline(tail_risk['cvar_95']*100, color='darkred', linestyle=':', linewidth=2, label='CVaR 95%')
ax1.set_xlabel('Daily Returns (%)')
ax1.set_ylabel('Density')
ax1.set_title('Return Distribution with Risk Measures')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Crash performance comparison
ax2 = axes[0, 1]
if crash_performance:
    crash_names = list(crash_performance.keys())
    crash_returns = [perf['total_return'] for perf in crash_performance.values()]
    crash_drawdowns = [perf['max_drawdown'] for perf in crash_performance.values()]
    
    x = np.arange(len(crash_names))
    width = 0.35
    
    ax2.bar(x - width/2, crash_returns, width, label='Total Return', alpha=0.8)
    ax2.bar(x + width/2, crash_drawdowns, width, label='Max Drawdown', alpha=0.8)
    
    ax2.set_xlabel('Market Crash Period')
    ax2.set_ylabel('Return (%)')
    ax2.set_title('Performance During Market Crashes')
    ax2.set_xticks(x)
    ax2.set_xticklabels(crash_names, rotation=45, ha='right')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# 3. Monte Carlo simulation results
ax3 = axes[1, 0]
percentiles = list(mc_stress['percentiles'].keys())
values = list(mc_stress['percentiles'].values())

ax3.plot(percentiles, values, marker='o', linewidth=2, markersize=6)
ax3.axhline(y=1, color='red', linestyle='--', alpha=0.7, label='Break-even')
ax3.fill_between(percentiles, values, 1, where=np.array(values) < 1, 
                alpha=0.3, color='red', label='Loss Region')
ax3.set_xlabel('Percentile')
ax3.set_ylabel('Portfolio Value')
ax3.set_title('Monte Carlo Simulation Results (1-Year)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Rolling maximum drawdown
ax4 = axes[1, 1]
cumulative_returns = (1 + portfolio_returns).cumprod()
rolling_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - rolling_max) / rolling_max
rolling_max_dd = drawdown.rolling(252).min()  # 1-year rolling max drawdown

ax4.fill_between(rolling_max_dd.index, rolling_max_dd*100, 0, alpha=0.7, color='red')
ax4.set_title('Rolling Maximum Drawdown (1-Year)')
ax4.set_ylabel('Max Drawdown (%)')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Risk budget analysis
print(f"\nRISK BUDGET ANALYSIS:")
print(f"-" * 30)

# Calculate individual asset contributions to portfolio risk
weights_series = pd.Series(recommended_weights)
cov_matrix = returns.cov() * 252
portfolio_variance = np.dot(weights_series.T, np.dot(cov_matrix, weights_series))
portfolio_volatility = np.sqrt(portfolio_variance)

marginal_contrib = np.dot(cov_matrix, weights_series) / portfolio_volatility
component_contrib = weights_series * marginal_contrib
risk_contrib_pct = component_contrib / component_contrib.sum() * 100

print(f"Portfolio Volatility: {portfolio_volatility*100:.2f}%")
print(f"\nRisk Contribution by Asset:")
for asset in assets:
    if asset in risk_contrib_pct.index:
        weight = weights_series[asset] * 100
        risk_contrib = risk_contrib_pct[asset]
        print(f"  {asset}: {weight:.1f}% weight, {risk_contrib:.1f}% risk contribution")

## Final Backtesting Summary and Recommendations

In [ ]:
# Generate comprehensive backtesting summary
print("\n" + "="*80)
print("COMPREHENSIVE BACKTESTING SUMMARY")
print("="*80)

# Best performing strategy
best_strategy_name = performance_df.iloc[0]['Strategy']
best_metrics = backtest_results[best_strategy_name]['metrics']

print(f"\n1. BEST PERFORMING STRATEGY: {best_strategy_name}")
print(f"-" * 50)
print(f"• Total Return: {best_metrics['Total Return']*100:.2f}%")
print(f"• Annualized Return: {best_metrics['Annualized Return']*100:.2f}%")
print(f"• Volatility: {best_metrics['Annualized Volatility']*100:.2f}%")
print(f"• Sharpe Ratio: {best_metrics['Sharpe Ratio']:.3f}")
print(f"• Maximum Drawdown: {best_metrics['Max Drawdown']*100:.2f}%")
print(f"• Information Ratio: {best_metrics['Information Ratio']:.3f}")
print(f"• Alpha vs SPY: {best_metrics['Alpha']*100:.2f}%")

# Strategy comparison insights
print(f"\n2. STRATEGY COMPARISON INSIGHTS:")
print(f"-" * 50)

# Compare optimized vs equal weight
if 'Optimized Portfolio' in backtest_results and 'Equal Weight' in backtest_results:
    opt_sharpe = backtest_results['Optimized Portfolio']['metrics']['Sharpe Ratio']
    eq_sharpe = backtest_results['Equal Weight']['metrics']['Sharpe Ratio']
    sharpe_improvement = ((opt_sharpe - eq_sharpe) / eq_sharpe) * 100
    
    opt_return = backtest_results['Optimized Portfolio']['metrics']['Annualized Return']
    eq_return = backtest_results['Equal Weight']['metrics']['Annualized Return']
    return_improvement = (opt_return - eq_return) * 100
    
    print(f"• Optimized vs Equal Weight:")
    print(f"  - Sharpe Ratio improvement: {sharpe_improvement:+.1f}%")
    print(f"  - Return improvement: {return_improvement:+.2f}% annually")

# Compare with traditional 60/40
if 'Optimized Portfolio' in backtest_results and '60/40 Traditional' in backtest_results:
    opt_sharpe = backtest_results['Optimized Portfolio']['metrics']['Sharpe Ratio']
    trad_sharpe = backtest_results['60/40 Traditional']['metrics']['Sharpe Ratio']
    
    opt_vol = backtest_results['Optimized Portfolio']['metrics']['Annualized Volatility']
    trad_vol = backtest_results['60/40 Traditional']['metrics']['Annualized Volatility']
    
    print(f"• Optimized vs 60/40 Traditional:")
    print(f"  - Sharpe Ratio: {opt_sharpe:.3f} vs {trad_sharpe:.3f}")
    print(f"  - Volatility: {opt_vol*100:.2f}% vs {trad_vol*100:.2f}%")

# Risk analysis summary
print(f"\n3. RISK ANALYSIS SUMMARY:")
print(f"-" * 50)
print(f"• Daily VaR (95%): {tail_risk['var_95']*100:.2f}%")
print(f"• Daily VaR (99%): {tail_risk['var_99']*100:.2f}%")
print(f"• Expected Shortfall (95%): {tail_risk['cvar_95']*100:.2f}%")
print(f"• Probability of annual loss: {mc_stress['probability_of_loss']:.1f}%")
print(f"• Worst-case scenario (1%): {(mc_stress['worst_case_1pct']-1)*100:.1f}% annual return")

# Market crash resilience
if crash_performance:
    avg_crash_return = np.mean([perf['total_return'] for perf in crash_performance.values()])
    avg_crash_dd = np.mean([perf['max_drawdown'] for perf in crash_performance.values()])
    
    print(f"\n4. MARKET CRASH RESILIENCE:")
    print(f"-" * 50)
    print(f"• Average return during crashes: {avg_crash_return:.2f}%")
    print(f"• Average max drawdown during crashes: {avg_crash_dd:.2f}%")
    print(f"• Number of crash periods analyzed: {len(crash_performance)}")

# Time period performance
print(f"\n5. TIME PERIOD PERFORMANCE:")
print(f"-" * 50)
if optimized_strategy in period_analysis:
    period_df = period_analysis[optimized_strategy]
    best_period = period_df.loc[period_df['Sharpe Ratio'].idxmax()]
    worst_period = period_df.loc[period_df['Sharpe Ratio'].idxmin()]
    
    print(f"• Best performing period: {best_period['Period']}")
    print(f"  - Sharpe Ratio: {best_period['Sharpe Ratio']:.3f}")
    print(f"  - Total Return: {best_period['Total Return']:.2f}%")
    
    print(f"• Worst performing period: {worst_period['Period']}")
    print(f"  - Sharpe Ratio: {worst_period['Sharpe Ratio']:.3f}")
    print(f"  - Total Return: {worst_period['Total Return']:.2f}%")

# Implementation insights
print(f"\n6. IMPLEMENTATION INSIGHTS:")
print(f"-" * 50)
print(f"• Rebalancing frequency tested: Monthly")
print(f"• Transaction costs not included (would reduce returns)")
print(f"• Tax implications not considered")
print(f"• Model assumes perfect execution")

# Key findings
print(f"\n7. KEY FINDINGS:")
print(f"-" * 50)

# Determine if optimization worked
optimization_success = False
if 'Optimized Portfolio' in backtest_results:
    opt_sharpe = backtest_results['Optimized Portfolio']['metrics']['Sharpe Ratio']
    
    # Compare with other strategies
    other_sharpes = [results['metrics']['Sharpe Ratio'] 
                    for name, results in backtest_results.items() 
                    if name != 'Optimized Portfolio']
    
    if opt_sharpe > max(other_sharpes):
        optimization_success = True

if optimization_success:
    print(f"✓ Portfolio optimization was successful")
    print(f"✓ Optimized portfolio outperformed benchmarks")
    print(f"✓ Risk-adjusted returns were maximized")
else:
    print(f"⚠ Portfolio optimization showed mixed results")
    print(f"⚠ Consider alternative optimization approaches")

# Check if forecasting helped
if portfolio_rec and 'TSLA' in recommended_weights:
    tsla_weight = recommended_weights['TSLA']
    if tsla_weight > 0.3:  # Significant TSLA allocation
        print(f"✓ Forecasting models influenced portfolio allocation")
        print(f"✓ TSLA allocation: {tsla_weight*100:.1f}% based on forecasts")
    else:
        print(f"⚠ Conservative TSLA allocation despite forecasts")

# Final recommendations
print(f"\n8. FINAL RECOMMENDATIONS:")
print(f"-" * 50)
print(f"• Implement {best_strategy_name} for optimal risk-adjusted returns")
print(f"• Rebalance monthly or when weights deviate >5%")
print(f"• Monitor portfolio during high volatility periods")
print(f"• Consider transaction costs in implementation")
print(f"• Review and update forecasts quarterly")
print(f"• Set stop-loss rules for individual positions if desired")
print(f"• Maintain emergency cash reserves outside this portfolio")

print(f"\n9. LIMITATIONS AND CAVEATS:")
print(f"-" * 50)
print(f"• Backtesting uses historical data (past ≠ future)")
print(f"• Transaction costs and slippage not included")
print(f"• Tax implications vary by investor")
print(f"• Model assumes rational markets and perfect execution")
print(f"• Forecasting accuracy may degrade over time")
print(f"• Correlation patterns may change in future")

print(f"\n" + "="*80)
print("BACKTESTING ANALYSIS COMPLETE")
print("="*80)

# Save backtesting results
backtesting_summary = {
    'best_strategy': best_strategy_name,
    'performance_comparison': performance_df.to_dict('records'),
    'stress_test_results': {
        'tail_risk': {k: v for k, v in tail_risk.items() if